In [1]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import pickle
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
items_num=max(max(user_dict.values()))
maxlen=1000
len_seq=100
batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=3
num_heads=1
interest_num=2
kernel_size=4
transformerencoder=0
multi_dense=0
neg_num=4
test_neg_num=100

In [4]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [5]:
'''train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)'''

'train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)\nprint(len(train_set),len(test_set),len(val_set),len(train_val_set))\nusers_num=len(test_set)'

In [6]:
'''users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*val_set)
val_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}'''

"users, click_seqs, pos_items,neg_items = zip(*train_set)\ntrain_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*test_set)\ntest_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*val_set)\nval_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}"

In [7]:
'''with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f, -1)
with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f, -1)
with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f, -1)'''

"with open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'wb') as f:\n    pickle.dump(train_data, f, -1)\nwith open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'wb') as f:\n    pickle.dump(test_data, f, -1)\nwith open('/root/autodl-tmp/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'wb') as f:\n    pickle.dump(val_data, f, -1)"

In [8]:
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'rb') as f:
    val_data=pickle.load(f)

In [9]:
print(np.shape(train_data['neg_item']),np.shape(test_data['neg_item']),np.shape(val_data['neg_item']))

(1267078, 4) (1411, 100) (1411, 4)


In [10]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

def normalize_adj_tensor(adj, seq_len):
    adj = adj + tf.expand_dims(tf.eye(seq_len), axis=0)
    rowsum = tf.reduce_sum(adj, axis=1)
    d_inv_sqrt = tf.pow(rowsum, -0.5)
    candidate_a = tf.zeros_like(d_inv_sqrt)
    d_inv_sqrt = tf.where(tf.math.is_inf(d_inv_sqrt), candidate_a, d_inv_sqrt)
    d_mat_inv_sqrt = tf.linalg.diag(d_inv_sqrt)
    norm_adg = tf.matmul(d_mat_inv_sqrt, adj)
    return norm_adg


def contrastive_label(seq_embed,att):
    s_item_emd,s_item_index=tf.math.top_k(att, k)
    s_item_index=tf.reshape(s_item_index,[-1,1])
    h_index=tf.sort(tf.tile(tf.reshape(tf.range(tf.shape(seq_embed)[0]),[-1,1]),[interest_num*k,1]),axis=0)
    index = tf.stack([h_index, s_item_index])
    index=tf.reshape(index,[-1,k,2])
    all_user_label=tf.gather_nd(seq_embed, index)
    all_user_label=tf.reshape(all_user_label,[-1,interest_num,k,hidden_size])
    all_user_label=tf.reduce_mean(all_user_label,axis=-2)
    return all_user_label

In [11]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

class gcn_layer(tf.keras.layers.Layer):
    def __init__(self, hidden_size, num_layer):
        super(gcn_layer, self).__init__()
        #weights_size_list = [hidden_size,64,64,64]
        #all_weights = {}
        self.num_layer=num_layer
        self.conv_dense=tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.01))
    
    def call(self,A, x):
    # gcn has three layers
        all_embeddings =[x]
        for k in range(self.num_layer):
            embeddings = tf.matmul(A, x)
            embeddings = self.conv_dense(embeddings)
            all_embeddings.append(embeddings)
        return all_embeddings
    
class hierarchical_interest(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(hierarchical_interest, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.conv_layer_dict={}
        for i in range(0,layers_num-1):
            self.conv_layer_dict['conv_layer_'+str(i)]=tf.nn.softmax(tf.Variable(initial_value=tf.random.truncated_normal([int((layers_num-1-i)*len_seq/(layers_num-1)),1,1,1],mean=0.5,stddev=1),name='w_'+str(i)),axis=0)

        self.conv_layer_dict['conv_layer_'+str(layers_num-1)]=tf.nn.softmax(tf.Variable(initial_value=tf.random.truncated_normal([1,1,1,1],mean=0.5,stddev=1),name='w_'+str(layers_num-1)),axis=0)
        self.layernorm = tf.keras.layers.LayerNormalization()
        
    def call(self, all_embeddings):
        all_embeddings[-1]=tf.nn.conv2d(tf.expand_dims(all_embeddings[-1],-1),self.conv_layer_dict['conv_layer_0'],[1,1,1,1],'SAME')
        all_embeddings[-1]=tf.squeeze(all_embeddings[-1],-1)
        for i,element in enumerate(all_embeddings):
            #print(i)
            if i==len(all_embeddings)-1:
                break
            all_embeddings[i]=tf.nn.conv2d(tf.expand_dims(element,-1),self.conv_layer_dict['conv_layer_'+str(layers_num-1-i)],[1,1,1,1],'SAME')
            all_embeddings[i]=tf.squeeze(all_embeddings[i],-1)
            all_embeddings[i]=all_embeddings[i]-scaled_dot_product_attention(all_embeddings[i],
                                                                             all_embeddings[-1],all_embeddings[-1],None)
            all_embeddings[i]=self.layernorm(all_embeddings[i])
        return all_embeddings

In [12]:
class mgnm(tf.keras.models.Model):
    def __init__(self):
        super(mgnm, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.pos_embedding = tf.keras.layers.Embedding(len_seq,hidden_size,input_length=1,
                                                       embeddings_initializer='random_normal',
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dropout = tf.keras.layers.Dropout(1-keep_rate)
        
        self.hiera=hierarchical_interest(hidden_size,0)
        
        self.multi_interest_dict={}
        if multi_dense:
            for i in range(0,layers_num+1):
                self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w1']=tf.keras.layers.Dense(hidden_size * 4, activation='tanh')
                self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w2']=tf.keras.layers.Dense(interest_num, activation=None)
        else:
            self.multi_interest_dict['multi_interest_layer_'+str(0)+'_w1']=tf.keras.layers.Dense(hidden_size * 4, activation='tanh')
            self.multi_interest_dict['multi_interest_layer_'+str(0)+'_w2']=tf.keras.layers.Dense(interest_num, activation=None)
        if transformerencoder:
            self.encoder_layer_dict={}
            for i in range(0,layers_num+1):
                self.encoder_layer_dict['encoder_layer_'+str(i)]=[TransformerEncoder(hidden_size, num_heads, 128,
                                                 (1-keep_rate), layer_norm_eps) for _ in range(blocks)]
        
        
        self.gcn=gcn_layer(hidden_size, layers_num)
        # norm
        self.use_l2norm = use_l2norm
        
    
    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        
        adj_l = tf.tile(tf.expand_dims(seq_embed, axis=2), [1, 1, len_seq, 1])
        adj_r = tf.tile(tf.expand_dims(seq_embed, axis=1), [1, len_seq, 1, 1])
        adj = tf.nn.sigmoid(tf.reduce_sum(adj_l * adj_r, axis=-1))
        self.adj_l1 = tf.norm(adj, ord=1)
        if 1:
            adj = normalize_adj_tensor(adj, len_seq) 
        all_embeddings=self.gcn(adj,seq_embed)
        all_embeddings=self.hiera(all_embeddings)
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.len_seq)), axis=0)  # (1, seq_len, embed_dim)
        att_outputs_dict={}
        for i,element in enumerate(all_embeddings):
            #print(i,'/',layers_num)
            att_outputs_dict['embed_'+str(i)]=element+pos_encoding
            att_outputs_dict['embed_'+str(i)] = self.dropout(att_outputs_dict['embed_'+str(i)])
        
        if transformerencoder:
            for i in range(0,layers_num+1):
                for block in self.encoder_layer_dict['encoder_layer_'+str(i)]:
                    att_outputs_dict['embed_'+str(i)] = block([att_outputs_dict['embed_'+str(i)],None])
        
        self.layer_interest_dict={}
        if multi_dense:
            for i in range(0,layers_num+1):
                att=self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w1'](att_outputs_dict['embed_'+str(i)])#[b,len_seq,4*h]
                att=self.multi_interest_dict['multi_interest_layer_'+str(i)+'_w2'](att)#[b,len_seq,num_heads]
                att=tf.transpose(att, [0, 2, 1])#[b,num_heads,len_seq]
                att=tf.nn.softmax(att)
                self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.matmul(att,att_outputs_dict['embed_'+str(i)])#[b,n,h]
                self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.reshape(self.layer_interest_dict['layer_'+str(i)+'_interest'],
                                                                                 [-1,1,interest_num,hidden_size])
        else:
            for i in range(0,layers_num+1):
                att=self.multi_interest_dict['multi_interest_layer_'+str(0)+'_w1'](att_outputs_dict['embed_'+str(i)])#[b,len_seq,4*h]
                att=self.multi_interest_dict['multi_interest_layer_'+str(0)+'_w2'](att)#[b,len_seq,num_heads]
                att=tf.transpose(att, [0, 2, 1])#[b,num_heads,len_seq]
                att=tf.nn.softmax(att)
                
                self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.matmul(att,att_outputs_dict['embed_'+str(i)])#[b,n,h]
                self.layer_interest_dict['layer_'+str(i)+'_interest']=tf.reshape(self.layer_interest_dict['layer_'+str(i)+'_interest'],
                                                                                 [-1,1,interest_num,hidden_size])
        

        mixed_interest=tf.concat([self.layer_interest_dict['layer_'+str(i)+'_interest'] for i in range(0,layers_num)],axis=1)#[b,l,n,h]
        
        
        #print(tf.shape(mixed_interest))
        if self.use_l2norm:
            pos_emb = tf.math.l2_normalize(pos_emb, axis=-1)
            neg_emb = tf.math.l2_normalize(neg_emb, axis=-1)
            mixed_interest = tf.math.l2_normalize(mixed_interest, axis=-1)
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))#[b,h]
        expanded_target_item=tf.expand_dims(tf.expand_dims(pos_emb,axis=1),axis=1)#[b,1,1,h]
        
        alpha=tf.matmul(expanded_target_item,tf.transpose(mixed_interest,[0,1,3,2]))#[b,l,1,n]
        alpha=tf.nn.softmax(alpha)
        res = tf.matmul(alpha,mixed_interest)#[b,l,1,h]
        pos_score=tf.reduce_max(tf.reduce_sum(tf.multiply(res,expanded_target_item),axis=-1),axis=-2)#[b,1]
            
        neg_emb=self.item_embedding(inputs['neg_item'])#[b,neg_num,h]
        expaned_neg_emb=tf.expand_dims(neg_emb,axis=1)#[b,1,neg_num,h]
        alpha=tf.matmul(expaned_neg_emb,tf.transpose(mixed_interest,[0,1,3,2]))#[b,l,neg_num,n]
        alpha=tf.nn.softmax(alpha)
        
        res = tf.matmul(alpha,mixed_interest)#[b,l,n,h]
        #expaned_neg_emb=tf.transpose(expaned_neg_emb,[0,2,1,3])#[b,l,n,h]
        neg_score=tf.reduce_max(tf.reduce_sum(tf.multiply(res,expaned_neg_emb),axis=-1),axis=-2)#[b,neg_num]
        
        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
        #pos_score = tf.tile(pos_score, [1, neg_score.shape[1]])
        
        loss = tf.reduce_mean(- tf.math.log(logits)[:,0])
        self.add_loss(loss)
        
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [13]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    print(np.shape(pred_y))
    return eval_rank(pred_y, metric_names, k)

In [14]:
'''callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = mgnm()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)

model.fit(
            x=train_data,
            epochs=100,
            batch_size=batch_size,
        validation_data=val_data,
    callbacks=[callback])
t2=time()
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (time() - t2,eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))'''

"callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)\nfrom time import time\nstrategy = tf.distribute.MirroredStrategy()\nbatch_size=batch_size*strategy.num_replicas_in_sync\nwith strategy.scope():\n    model = mgnm()\n    model.summary()\n    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)\n    model.compile(optimizer=optimizer)\n\nmodel.fit(\n            x=train_data,\n            epochs=100,\n            batch_size=batch_size,\n        validation_data=val_data,\n    callbacks=[callback])\nt2=time()\neval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)\nprint('Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'\n              % (time() - t2,eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))"

In [15]:
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = mgnm()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)
best_eval_dict={'hr':0.,'mrr':0.,'ndcg':0.}
eval_patience={'hr':0.,'mrr':0.,'ndcg':0.}
for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size,
        validation_data=val_data)
    t2 = time()
    eval_dict = eval_pos_neg(model, val_data, ['hr', 'mrr', 'ndcg'], 1, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
    for i in eval_dict.keys():
        if eval_dict[i]>=best_eval_dict[i]:
            best_eval_dict[i]=eval_dict[i]
        else:
            eval_patience[i]+=1
    if (np.array(list(eval_patience.values()))>=5).any():
        print('Trigger early stopping')
        break
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))   

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
tf.reshape_11 (TFOpLambda)      (None,)              0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 100)]        0                                            
_______________________________________________

In [16]:
#maxlen=1000
#len_seq=100
#batch_size=512
#epoch_num=100
#hidden_size=64
#keep_rate=0.9
#layers_num=10
#num_heads=1
#interest_num=3
#kernel_size=4
#transformerencoder=0
#neg_num=4
#test_neg_num=100
#Iteration 10 Fit [58.3 s], Evaluate [0.3 s]: HR = 0.2941, MRR = 0.0970, NDCG = 0.1424

In [17]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

'def train_model(model,ds_train,ds_valid,epoches):\n\n    for epoch in tf.range(1,epoches+1):\n        model.reset_metrics()\n        \n        # 在后期降低学习率\n        if epoch == 5000:\n            model.optimizer.lr.assign(model.optimizer.lr/2.0)\n            tf.print("Lowering optimizer Learning Rate...\n\n")\n        \n        for x, y in ds_train:\n            train_result = model.train_on_batch(x, y)\n\n        for x, y in ds_valid:\n            valid_result = model.test_on_batch(x, y,reset_metrics=False)\n            \n        if epoch%100 ==0:\n            tf.print("epoch = ",epoch)\n            print("train:",dict(zip(model.metrics_names,train_result)))\n            print("valid:",dict(zip(model.metrics_names,valid_result)))\n            print("")'

In [18]:
'''train_model(model,ds_train,ds_test,800)'''

'train_model(model,ds_train,ds_test,800)'